In [ ]:
import uproot3
import numpy as np
from tqdm import tqdm as tqdm
import matplotlib.pyplot as plt
import os
import os.path as osp
import awkward0
import math
import torch_geometric
import torch
from torch_geometric.data import Data
import glob

In [ ]:
filename = '/data_cms_upgrade/motta/HGCAL_SKIMS/SKIM_31May2021/SKIM_RelValTenTau_PU200_caloTruth/mergedOutput.root'
test = uproot3.open(filename)['HGCALskimmedTree']
print(test.numentries)
#test.keys()

In [ ]:
### arrify only required vars

cl3d_iTau = test['cl3d_iTau'].array()
cl3d_pt = test['cl3d_pt'].array()
cl3d_energy = test['cl3d_energy'].array()
cl3d_eta = test['cl3d_eta'].array()
cl3d_phi = test['cl3d_phi'].array()
cl3d_clusters_n = test['cl3d_clusters_n'].array()
cl3d_clusters_id = test['cl3d_clusters_id'].array()
cl3d_layer_pt = test['cl3d_layer_pt'].array()
cl3d_showerlength = test['cl3d_showerlength'].array()
cl3d_coreshowerlength = test['cl3d_coreshowerlength'].array()
cl3d_firstlayer = test['cl3d_firstlayer'].array()
cl3d_maxlayer = test['cl3d_maxlayer'].array()
cl3d_seetot = test['cl3d_seetot'].array()
cl3d_seemax = test['cl3d_seemax'].array()
cl3d_spptot = test['cl3d_spptot'].array()
cl3d_sppmax = test['cl3d_sppmax'].array()
cl3d_szz = test['cl3d_szz'].array()
cl3d_srrtot = test['cl3d_srrtot'].array()
cl3d_srrmax = test['cl3d_srrmax'].array()
cl3d_srrmean = test['cl3d_srrmean'].array()
#cl3dpt = test['cl3d_emaxe'].array()
cl3d_hoe = test['cl3d_hoe'].array()
cl3d_meanz = test['cl3d_meanz'].array()
cl3d_layer10 = test['cl3d_layer10'].array()
cl3d_layer50 = test['cl3d_layer50'].array()
cl3d_layer90 = test['cl3d_layer90'].array()
cl3d_ntc67 = test['cl3d_ntc67'].array()
cl3d_ntc90 = test['cl3d_ntc90'].array()

In [ ]:
evskip = 0
for evt in tqdm(range(cl3d_iTau.size),desc='events processed'):
    if len(np.unique(cl3d_iTau[evt],return_counts=True)[1])==1:
        evskip += 1
        continue
     

    feats = np.stack((            
            cl3d_eta[evt],
            cl3d_phi[evt],
            cl3d_pt[evt],
            cl3d_energy[evt],
            cl3d_clusters_n[evt],
            cl3d_showerlength[evt],
            cl3d_coreshowerlength[evt],
            cl3d_firstlayer[evt],
            cl3d_maxlayer[evt],
            cl3d_seetot[evt],
            cl3d_seemax[evt],
            cl3d_spptot[evt],
            cl3d_sppmax[evt],
            cl3d_szz[evt],
            cl3d_srrtot[evt],
            cl3d_srrmax[evt],
            cl3d_srrmean[evt],
            cl3d_hoe[evt],
            cl3d_meanz[evt],
            cl3d_layer10[evt],
            cl3d_layer50[evt],
            cl3d_layer90[evt],
            cl3d_ntc67[evt],
            cl3d_ntc90[evt],
    
                    )).T

    ### label taus as 1 rest as 0
    labelvec = [1 if (i>=0) else 0   for i in cl3d_iTau[evt]]

    
    
    raw_dir='/grid_mnt/data__data.polcms/cms/sghosh/TAUGNN/'
    processed_dir = raw_dir + 'GRAPH_TENTAU/'
    
    
    import os
    import os.path as osp
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)
    
    
    ## save data as graphs
    torch.save(Data(x = torch.tensor(feats, dtype=torch.float32),
               y=torch.tensor(labelvec,dtype=torch.long)),
    #            en = torch.tensor(trkgunen,dtype=torch.float32)),
                osp.join(processed_dir, 'data_{}.pt'.format(evt)))
                       
             
    
    
    
    
print('events with no cluster-tau match:',evskip)
    